In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
ls

'new split dataset 75:14:1'/   test/   train/   valid/


In [5]:
cd drive/MyDrive/DATASET/skin-lesions/

/content/drive/MyDrive/DATASET/skin-lesions


In [46]:
train_dir = "train"
test_dir = "test"
valid_dir = "valid"

In [10]:
!pip install split-folders

In [24]:
import splitfolders  # or import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("train", output="new split dataset 75:14:1", seed=1337, ratio=(.75, .24, .01), group_prefix=None) # default value


Copying files: 0 files [00:00, ? files/s]
Copying files: 6 files [00:00, 46.67 files/s]
Copying files: 11 files [00:01,  6.74 files/s]
Copying files: 14 files [00:03,  3.23 files/s]
Copying files: 16 files [00:05,  2.16 files/s]
Copying files: 17 files [00:05,  2.09 files/s]
Copying files: 18 files [00:06,  2.16 files/s]
Copying files: 19 files [00:06,  2.28 files/s]
Copying files: 20 files [00:06,  2.43 files/s]
Copying files: 21 files [00:07,  1.97 files/s]
Copying files: 22 files [00:08,  1.73 files/s]
Copying files: 23 files [00:09,  1.62 files/s]
Copying files: 24 files [00:10,  1.47 files/s]
Copying files: 25 files [00:10,  1.73 files/s]
Copying files: 26 files [00:10,  2.02 files/s]
Copying files: 27 files [00:11,  2.22 files/s]
Copying files: 28 files [00:11,  2.40 files/s]
Copying files: 29 files [00:11,  2.19 files/s]
Copying files: 30 files [00:12,  2.12 files/s]
Copying files: 31 files [00:12,  2.34 files/s]
Copying files: 32 files [00:13,  1.89 files/s]
Copying files: 33 

In [47]:
ls

'new split dataset 75:14:1'/   test/   train/   valid/


In [48]:
ls "new split dataset 75:14:1"

test/  train/  val/


In [49]:
train_path = "new split dataset 75:14:1/train"
test_path  = "new split dataset 75:14:1/test"
val_path   = "new split dataset 75:14:1/val"

In [50]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [51]:
training_set = train_datagen.flow_from_directory(  train_path,
                                       target_size=(128, 128),
                                             color_mode="rgb",
                                                batch_size=32,
                                     class_mode='categorical',
                                               shuffle = True)

Found 1499 images belonging to 3 classes.


In [52]:
validation_set = val_datagen.flow_from_directory(    val_path,
                                       target_size=(128, 128),
                                             color_mode="rgb",
                                                batch_size=32,
                                     class_mode='categorical',
                                               shuffle = False)

Found 478 images belonging to 3 classes.


In [53]:
test_set = val_datagen.flow_from_directory(         test_path,
                                       target_size=(128, 128),
                                             color_mode="rgb",
                                                batch_size=32,
                                     class_mode='categorical',
                                               shuffle = False)

Found 23 images belonging to 3 classes.


In [54]:
# IMPORT SEMUA PAKET YANG ANDA PERLUKAN UNTUK MENDEFINISIKAN MODEL DISINI
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Activation, Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [55]:
# DEFINISIKAN MODEL ANDA DISINI
model = Sequential()

model.add(InputLayer(input_shape=[128,128,3]))

model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Conv2D(filters=3, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())

# Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [56]:
# TULIS CODE UNTUK MENAMPILKAN SUMMARY DARI ARSITEKTUR MODEL ANDA DISINI
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 128, 128, 16)      448       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 64, 32)        4640      
_________________________________________________________________
batch_normalization_4 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 16, 16, 64)       

In [57]:
# TULIS CODE UNTUK DEFINISI LEARNING RATE DISINI 
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.01), 
              loss='categorical_crossentropy', 
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [58]:
# TULIS CODE UNTUK TRAINING DISINI
import time

start = time.time()
H = model.fit_generator(
        training_set,
        steps_per_epoch=44,
        epochs=100,
        validation_data=validation_set,
        validation_steps=5)

KeyboardInterrupt: ignored

In [ ]:
from keras.models import save_model
model = save_model(model,'modul3.h5')

In [ ]:
!ls

In [ ]:
# TULIS CODE UNTUK PLOT EVALUASI MODEL DISINI
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.title("Loss Plot")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_acc"], label="val_acc")
plt.title("Accuracy Plot")
plt.xlabel("Epoch #")
plt.ylabel("Acc")
plt.legend()
plt.show()

In [ ]:
pred2 = model2.predict(x_val)
labels2 = (pred2 > 0.5).astype(np.int)

print(classification_report(y_val, labels2))